In [ ]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.utils import np_utils
import numpy as np
seed = 7
np.random.seed(seed)
(X_train,y_train),(X_test,y_test)=mnist.load_data()
X_train = X_train.reshape((-1,28*28)).T
X_test = X_test.reshape((-1,28*28)).T
X_train = X_train / 255
X_test = X_test / 255
y_train = np_utils.to_categorical(y_train)
y_train= y_train.T
y_test = np_utils.to_categorical(y_test)
y_test= y_test.T
print(y_train.shape)

In [ ]:
#important
def initialize_parameters_deep(layer_dims):
    np.random.seed(3)
    parameters={}
    L=len(layer_dims)
    for l in range(1,L):
        parameters['W'+str(l)]= np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b'+str(l)]= np.zeros((layer_dims[l],1))
    return parameters

In [ ]:
#important
import numpy as np
import h5py
import matplotlib.pyplot as plt
def linear_forward(A,W,b):
    #A là các activations (đầu vào) của lớp trước đó
    #W là các trọng số (weights) của lớp hiện tại
    #b là các bias của lớp hiện tại
    Z=np.dot(W,A)+b
    cache=(A,W,b)
    return Z,cache

In [ ]:
def sigmoid(Z):
    A=1/(1+np.exp(-Z))
    cache=Z
    return A,cache
def sigmoid_backward(dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    assert (dZ.shape == Z.shape)
    return dZ
def relu(Z):
    A = np.maximum(0,Z)
    cache = Z
    return A , cache
def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True) 
    dZ[Z <= 0] = 0
    assert (dZ.shape == Z.shape)
    return dZ
def softmax(x):
  e = np.exp(x)
  return e / np.sum(e)
def softmax_backward(dA,cache):
    Z=cache
    _s = cache.reshape(-1, 1)
    dZ=np.multiply(np.diagonal(np.diagflat(_s) - np.dot(_s, _s.T)),dA)
    assert (dZ.shape == Z.shape)
    return dZ

In [ ]:
#important

def linear_activation_forward(A_pre,W,b,activation):
    if(activation=='sigmoid'):
        Z,linear_cache = linear_forward(A_pre,W,b)
        A,activation_cache=sigmoid(Z)
    elif(activation=='relu'):
        Z,linear_cache = linear_forward(A_pre,W,b)
        A,activation_cache=relu(Z)
    cache=(linear_cache,activation_cache)
    return A, cache

In [ ]:
#important
def L_model_forward(X,paramters):
    caches=[]
    A=X
    L=len(paramters)
    for l in range(1,L):
        A_pre=A
        A, cache =linear_activation_forward(A_pre,paramters['W'+str(l)],paramters['b'+str(l)],'relu')
        caches.append(cache)
    AL, cache= linear_activation_forward(A_pre,paramters['W'+str(L)],paramters[b+str(L)],'softmax')
    caches.append(cache)
    assert(AL.shape == (1,X.shape[1]))
    return AL, caches

In [ ]:
def compute_cost(AL,Y):
    m=Y.shape[1]
#     cost=-(1/m)*np.sum(np.multiply(Y,np.log(AL))+np.multiply((1-Y),np.log(1-AL)))
    cost=-(1/m)*np.sum(np.sum(np.multiply(Y,np.log(AL))))
    cost= np.squeeze(cost)
    return cost

In [ ]:
#important
def linear_backward(dZ,cache):
    A_pre,W,b = cache
    m=A_pre.shape[1]
    dW=(1/m)*np.dot(dZ,np.transpose(A_pre))
    db=(1/m)*np.sum(dZ,axis=1,keepdims=True)
    dA_pre=np.dot(np.transpose(W),dZ)
    return dA_pre,dW,db

In [ ]:
#important
def linear_activation_backward(dA,cache,activation):
    linear_cache,activation_cache= cache
    if activation=="relu":
        dZ=relu_backward(dA,activation_cache)
#         print(dZ.shape)
        dA_pre,dW,db=linear_backward(dZ,linear_cache)
    elif activation=="sigmoid":
        dZ=sigmoid_backward(dA,activation_cache)
#         print(dZ.shape)        
        dA_pre,dW,db=linear_backward(dZ,linear_cache)
    elif activation=="softmax":
        dZ=softmax_backward(dA,activation_cache)
#         print(dZ.shape)        
        dA_pre,dW,db=linear_backward(dZ,linear_cache)
    return dA_pre,dW,db

In [ ]:
#important
def L_model_backward(AL,Y,caches):
    grades={}
    L= len(caches)
    m= AL.shape[1]
    Y=Y.reshape(AL.shape)
    dAL=-(np.divide(Y,AL)-np.divide(1-Y,1-AL))
    current_cache=caches[L-1]
#     grades['dA'+str(L)],grades['dW'+str(L)],grades['db'+str(L)]=linear_activation_backward(dAL,current_cache,'sigmoid')
    grades['dA'+str(L)],grades['dW'+str(L)],grades['db'+str(L)]=linear_activation_backward(dAL,current_cache,'softmax')
    for l in reversed(range(L-1)):
        current_cache=caches[l]
        dA_pre_temp,dW_temp,db_temp= linear_activation_backward(grades['dA'+str(l+2)],current_cache,'relu')
        grades['dA'+str(l+1)]=dA_pre_temp
        grades['dW'+str(l+1)]=dW_temp
        grades['db'+str(l+1)]=db_temp
    return grades  

In [ ]:
# important
def update_parameters(parameters,grads,learning_rate):
    L = len(parameters) // 2 # number of layers in the neural network
    for l in range(L):
        parameters["W" + str(l+1)] =parameters["W" + str(l+1)]-learning_rate*grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] =parameters["b" + str(l+1)]-learning_rate*grads["db" + str(l+1)]
    return parameters


In [ ]:
def two_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):

    np.random.seed(1)
    grads = {}
    costs = []                              # to keep track of the cost
    m = X.shape[1]                           # number of examples
    (n_x, n_h, n_y) = layers_dims

    parameters = initialize_parameters_deep([n_x, n_h, n_y])

    
    # Get W1, b1, W2 and b2 from the dictionary parameters.
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # Loop (gradient descent)

    for i in range(0, num_iterations):

        # Forward propagation: LINEAR -> RELU -> LINEAR -> SOFTMAX. Inputs: "X, W1, b1". Output: "A1, cache1, A2, cache2".
        ### START CODE HERE ### (≈ 2 lines of code)
        A1, cache1 = linear_activation_forward(X, W1, b1, "relu")
        A2, cache2 = linear_activation_forward(A1, W2, b2, "softmax")
        ### END CODE HERE ###
        
        # Compute cost
        ### START CODE HERE ### (≈ 1 line of code)
        cost = compute_cost(A2, Y)
        ### END CODE HERE ###
        
        # Initializing backward propagation
        dA2 = - (np.divide(Y, A2) - np.divide(1 - Y, 1 - A2))
        
        # Backward propagation. Inputs: "dA2, cache2, cache1". Outputs: "dA1, dW2, db2; also dA0 (not used), dW1, db1".
        ### START CODE HERE ### (≈ 2 lines of code)
        dA1, dW2, db2 = linear_activation_backward(dA2, cache2, "sofmax")
        dA0, dW1, db1 = linear_activation_backward(dA1, cache1, "relu")
        ### END CODE HERE ###
        
        # Set grads['dWl'] to dW1, grads['db1'] to db1, grads['dW2'] to dW2, grads['db2'] to db2
        grads['dW1'] = dW1
        grads['db1'] = db1
        grads['dW2'] = dW2
        grads['db2'] = db2
        
        # Update parameters.
        ### START CODE HERE ### (approx. 1 line of code)
        parameters = update_parameters(parameters, grads, learning_rate)
        ### END CODE HERE ###

        # Retrieve W1, b1, W2, b2 from parameters
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]
        
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))

        if print_cost and i % 100 == 0:
            costs.append(cost)
       
    # plot the cost

    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

In [ ]:
parameters = two_layer_model(X_train, y_train, layers_dims = (784, 16, 10), num_iterations = 2500, print_cost=True)